In [1]:
from pyedflib import EdfReader

class EdfReaderWrapper(EdfReader):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    def __enter__(self):
        return self
    def __exit__(self, *args):
        super().close()

In [ ]:
import os
import json
import datetime

outdir = "./chbmit-plotinfo"
data_dir = "/home/zgl/publicdata/CHBMIT/0_data_edf/chbmit/1.0.0"
ignore_lst = ["chb16_18.edf", "chb16_19.edf", "chb17c_13.edf", "chb18_01.edf", "chb19_01.edf", "chb11_01.edf", "chb12_27.edf", "chb12_28.edf", "chb12_29.edf", "chb09_01.edf", "chb15_01.edf"]

# TODO 改为上面的isoformat
dt_fmt = '%Y-%m-%d %H:%M:%S'

for indexPatient in range(1, 24+1):
    result_obj = {"record_lst": [], "seizure_lst": [], "unused_rec_idx_lst": []}
    with open(os.path.join(data_dir, f"chb{indexPatient:02d}", f"chb{indexPatient:02d}-summary.txt"), 'r') as f:
        for line in f:
            data=line.split(':')
            if(data[0]=="File Name"):
                edfName=data[1].strip()

                with EdfReaderWrapper(os.path.join(data_dir, f"chb{indexPatient:02d}", edfName)) as pedf: 
                    startTime = pedf.getStartdatetime()
                    startStr = startTime.isoformat()
                    endTime = startTime + datetime.timedelta(seconds=pedf.getFileDuration())
                    endStr = endTime.isoformat()
                    result_obj["record_lst"].append({
                        "file": edfName, 
                        "span": [startStr, endStr], 
                        "info": f"edfName records {startTime} ~ {endTime} \r\n of shape {pedf.signals_in_file, pedf.getNSamples()[0]}"
                    })
                    
                nextLine = f.readline().strip()
                while (len(nextLine) != 0 and not nextLine.startswith('Number of Seizures in File:')):
                    nextLine = f.readline().strip()
                if nextLine.startswith('Number of Seizures in File:'):
                    for j in range(0, int(nextLine.split(':')[1])):
                        szStartSec = int(f.readline().split(': ')[1].strip().split(' ')[0])
                        szEndSec = int(f.readline().split(': ')[1].strip().split(' ')[0])
                        result_obj["seizure_lst"].append({
                            "span": [startTime+timedelta(seconds=szStartSec), startTime+timedelta(seconds=szEndSec)], 
                            "info": f'Onset {startTime+timedelta(seconds=szStartSec)}, last {szEndSec - szStartSec}s'
                        })

        # result_obj["record_lst"].sort(key=lambda obj:obj["span"])
        # result_obj["seizure_lst"].sort(key=lambda obj:obj["span"])
        for k, rec_info in enumerate(result_obj["record_lst"]):
            if rec_info["file"] in ignore_lst:
                result_obj["unused_rec_idx_lst"].append(k)
        #   rec_info["span"] = [rec_info["span"][0].isoformat(), rec_info["span"][1].isoformat()]
        
    with open(os.path.join(outdir, f'chb{indexPatient:02d}.json'), 'wt') as f:
        json.dump(result_obj, f, indent=2)

In [18]:
with open("./cui-mneinfo/cuiyibing_converted.json", "rt") as f:
    result_obj = json.load(f)

for k, rec_info in enumerate(result_obj["record_lst"]):
    # if rec_info["file"] in ignore_lst:
    #     result_obj["unused_rec_idx_lst"].append(k)
    rec_info["span"] = [datetime.fromisoformat(rec_info["span"][0]), datetime.fromisoformat(rec_info["span"][1])]

for seiz_info in result_obj["seizure_lst"]:
    seiz_info["span"] = [datetime.fromisoformat(seiz_info["span"][0]), datetime.fromisoformat(seiz_info["span"][1])]


In [ ]:
import plotly.offline as pyo
from timeline import get_pat_timeline
fig = get_pat_timeline(title="Cui YiBing", record_seq=result_obj["record_lst"], seizure_seq=result_obj["seizure_lst"])

pyo.plot(fig, filename="./cui-mneinfo/cui-timeline.html", # include_plotlyjs="./plotly.min.js", 
            auto_open=False, image='svg', image_width=2560, image_height=1440)
fig.show()